In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import time

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

from dataio import getdata, writesub

In [2]:
# /media/ryan/Charlemagne/kaggle/santander-kaggle
trainpath = '/media/ryan/Charlemagne/kaggle/santander-kaggle/train.csv' 
testpath = '/media/ryan/Charlemagne/kaggle/santander-kaggle/test.csv' 
df_train,df_test = getdata(trainpath,testpath)

In [3]:
df_features = pd.read_csv('features/feature_combos_15.csv')
df_features = df_features.drop('Unnamed: 0',axis=1)
df_features.head(5)

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,combinedAUC
0,saldo_var30,var15,saldo_medio_var5_ult1,num_meses_var5_ult3,num_var30,saldo_medio_var5_hace3,ind_var5,var38,num_var45_hace2,num_var12_0,num_var39_0,num_var42_0,num_var8_0,num_var45_hace3,imp_op_var39_efect_ult1,0.837312
1,saldo_var30,var15,saldo_medio_var5_hace2,num_var42,ind_var5,var38,num_meses_var39_vig_ult3,num_var12_0,ind_var13,num_var39_0,num_var42_0,ind_var8_0,num_var45_ult3,imp_op_var41_efect_ult1,num_var13_corto,0.836649
2,saldo_var30,var15,saldo_medio_var5_hace2,num_meses_var5_ult3,ind_var30,var38,num_meses_var39_vig_ult3,ind_var41_0,imp_op_var41_ult1,ind_var12_0,ind_var13,num_var42_0,num_var45_hace3,num_var45_ult3,num_var13_corto,0.836278
3,saldo_var30,var15,saldo_medio_var5_hace2,num_var4,saldo_medio_var5_hace3,num_var42,ind_var5,var38,num_meses_var39_vig_ult3,imp_op_var41_ult1,ind_var12_0,num_var41_0,num_var13_0,num_var42_0,imp_op_var41_efect_ult1,0.836258
4,saldo_var30,var15,saldo_medio_var5_ult3,num_var42,num_var5,ind_var5,var36,var38,num_meses_var39_vig_ult3,ind_var39_0,num_var12_0,ind_var8_0,num_var45_hace3,imp_op_var39_efect_ult1,ind_var43_recib_ult1,0.836237


In [4]:
df_features[df_features['combinedAUC']>0.835]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,combinedAUC
0,saldo_var30,var15,saldo_medio_var5_ult1,num_meses_var5_ult3,num_var30,saldo_medio_var5_hace3,ind_var5,var38,num_var45_hace2,num_var12_0,num_var39_0,num_var42_0,num_var8_0,num_var45_hace3,imp_op_var39_efect_ult1,0.837312
1,saldo_var30,var15,saldo_medio_var5_hace2,num_var42,ind_var5,var38,num_meses_var39_vig_ult3,num_var12_0,ind_var13,num_var39_0,num_var42_0,ind_var8_0,num_var45_ult3,imp_op_var41_efect_ult1,num_var13_corto,0.836649
2,saldo_var30,var15,saldo_medio_var5_hace2,num_meses_var5_ult3,ind_var30,var38,num_meses_var39_vig_ult3,ind_var41_0,imp_op_var41_ult1,ind_var12_0,ind_var13,num_var42_0,num_var45_hace3,num_var45_ult3,num_var13_corto,0.836278
3,saldo_var30,var15,saldo_medio_var5_hace2,num_var4,saldo_medio_var5_hace3,num_var42,ind_var5,var38,num_meses_var39_vig_ult3,imp_op_var41_ult1,ind_var12_0,num_var41_0,num_var13_0,num_var42_0,imp_op_var41_efect_ult1,0.836258
4,saldo_var30,var15,saldo_medio_var5_ult3,num_var42,num_var5,ind_var5,var36,var38,num_meses_var39_vig_ult3,ind_var39_0,num_var12_0,ind_var8_0,num_var45_hace3,imp_op_var39_efect_ult1,ind_var43_recib_ult1,0.836237
5,saldo_var30,var15,saldo_var42,num_var4,saldo_medio_var5_ult1,saldo_var5,num_meses_var5_ult3,num_var5,var38,ind_var13,ind_var13_0,num_var42_0,num_var8_0,imp_op_var39_efect_ult1,imp_op_var41_efect_ult1,0.836178
6,var15,saldo_medio_var5_hace2,saldo_var42,num_var30,var36,var38,num_var30_0,ind_var41_0,ind_var12_0,num_var13,ind_var13,imp_op_var39_efect_ult1,num_var5_0,num_var13_corto,ind_var43_recib_ult1,0.836169
7,saldo_var30,var15,num_var4,num_meses_var5_ult3,num_var42,num_var5,var36,var38,num_meses_var39_vig_ult3,imp_op_var41_ult1,num_var12_0,num_var41_0,num_var42_0,ind_var13_corto,saldo_var13_corto,0.836063
8,saldo_var30,var15,saldo_medio_var5_ult3,num_var30,var36,var38,num_var45_hace2,num_var30_0,imp_op_var39_ult1,num_var42_0,ind_var8_0,num_var45_hace3,imp_op_var39_efect_ult1,imp_op_var41_efect_ult1,saldo_var13_corto,0.835898
9,saldo_var30,var15,saldo_medio_var5_hace2,num_var5,ind_var5,var38,num_var30_0,ind_var39_0,imp_op_var39_ult1,saldo_var13,num_var39_0,num_var42_0,num_var45_ult3,num_var5_0,saldo_medio_var13_corto_ult1,0.835891


In [5]:
start_time = time.time()

In [6]:
rs = 19683

# split data into train and test
test_id = df_test.ID
test = df_test.drop(["ID"],axis=1)

X = df_train.drop(["TARGET","ID"],axis=1)
y = df_train.TARGET.values

X_train = X
y_train = y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=rs)

In [7]:
thresh = 0.835
df_features_greater_than = df_features[df_features['combinedAUC']>thresh]
sub = np.zeros((test.shape[0],df_features_greater_than.shape[0]))
for index, row in df_features_greater_than.iterrows():
    # grid search for params
    xgb_model = xgb.XGBClassifier()
    clf = GridSearchCV(xgb_model, {'max_depth': [2, 6, 10],
                                   'learning_rate': [0.01, 0.1, 0.25],
                                   'n_estimators': [50, 100, 200],
                                   'seed':[rs]}, 
                        verbose=0, n_jobs=-1, scoring = 'roc_auc')
    clf.fit(X_train, y_train)

    ## # Train Model
    # classifier from xgboost
    sub[:,index] = clf.predict_proba(test)[:,1]

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed: 27.7min finished


ValueError: could not broadcast input array from shape (75818,2) into shape (75818)

In [ ]:
submissions = pd.DataFrame(sub)
submissions['ID'] = pd.Series(test_id)
submissions.to_csv('features15.xgb.probs.csv')